<a href="https://colab.research.google.com/github/2003Yash/self-attention/blob/main/Self_Attention_for_Transformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

***
self attention: we use q,k,v vectors to create attention matrix (input_len x input_len) and we multiply them and use softmax to get attention matrix for word context inportance with other words in how much each word should look into other words in the same sentence

In [ ]:
import numpy as np
import math

L, d_k, d_v = 4, 8, 8 # l = length of string and size if each vector = 8(no specific reason for 8)
q = np.random.randn(L, d_k)
k = np.random.randn(L, d_k)
v = np.random.randn(L, d_v)
#all 8x1 vectors randomly assigned values

In [ ]:
print("Q\n", q)
print("K\n", k)
print("V\n", v)

Q
 [[-1.79708796  0.52122516  0.34239011 -0.2192233   0.95937318  0.01950385
  -0.41948086  0.19684975]
 [-0.28763054  1.56560955 -1.50161045  0.3472905  -0.31556079  1.35173172
  -1.14744159  0.26172031]
 [ 0.49501539  0.83643826 -0.61257021 -0.46052311  0.21198769 -1.21393321
   0.68011574  1.08998828]
 [ 1.04793736  0.90690933  1.12068841  0.96005338 -0.48338012 -0.49387283
   0.89347304 -0.98213068]]
K
 [[-1.69457653  1.14189547 -0.54117259 -0.21483296 -1.06478926 -2.33830904
  -0.53044194 -2.02805418]
 [-0.98502234  0.65551073  0.3332141  -0.5899652  -0.47458067 -1.74058004
  -1.61271634  0.83231212]
 [ 0.5630061  -0.12879049 -0.89508148  1.07619482  2.16600192  0.2716528
  -0.04767808  2.50152317]
 [ 0.13802501  0.90824179 -0.63588164 -2.5203722   0.31566198  0.4199465
  -1.52188367 -1.18428275]]
V
 [[-0.2684259   0.74669319  0.25532415  0.54156151  0.12737344  0.85743961
  -0.56087513 -0.44426863]
 [-0.04093924  1.80102587 -0.27960843 -1.57039158  2.52074117 -0.07819322
   0.892

In [ ]:
np.matmul(q, k.T)
#4x4 aresult since the sequence is 4x4 and it describes attendtion of each other word in sentence

array([[ 2.25844396,  2.7063599 ,  0.97443233,  1.27646365],
       [ 0.26630277,  0.46964187,  1.74734626,  3.36616325],
       [ 0.58823693,  1.95098796,  3.04726077, -0.39055741],
       [ 1.63445254, -1.80005162, -3.17729507, -2.72060903]])

In [ ]:
q.var(), k.var(), np.matmul(q, k.T).var()
#since varience is too large for thrid value we divide it with sq_rt(dimension size) to bring all variances in a reasonable range

(0.7373171315861786, 1.4392136315212765, 3.586251586314168)

In [ ]:
scaled = np.matmul(q, k.T) / math.sqrt(d_k)
q.var(), k.var(), scaled.var()

(0.7373171315861786, 1.4392136315212765, 0.44828144828927097)

In [ ]:
scaled # normalized vector of attention of other words in the line

array([[ 0.79848052,  0.95684272,  0.34451385,  0.45129805],
       [ 0.09415225,  0.16604347,  0.61778019,  1.19011843],
       [ 0.20797316,  0.68977841,  1.07736938, -0.1380829 ],
       [ 0.57786624, -0.63641435, -1.12334345, -0.96188055]])

In [ ]:
# we need mask so decoder won't focus on next since we don't even have the next we still nave to predict it
mask = np.tril(np.ones( (L, L) ))
mask

array([[1., 0., 0., 0.],
       [1., 1., 0., 0.],
       [1., 1., 1., 0.],
       [1., 1., 1., 1.]])

In [ ]:
mask[mask == 0] = -np.infty
mask[mask == 1] = 0
# since after adding mask to scaled self attention vector 0 will do no change and -int will always be -int

In [ ]:
mask

array([[  0., -inf, -inf, -inf],
       [  0.,   0., -inf, -inf],
       [  0.,   0.,   0., -inf],
       [  0.,   0.,   0.,   0.]])

In [ ]:
scaled + mask

array([[ 0.79848052,        -inf,        -inf,        -inf],
       [ 0.09415225,  0.16604347,        -inf,        -inf],
       [ 0.20797316,  0.68977841,  1.07736938,        -inf],
       [ 0.57786624, -0.63641435, -1.12334345, -0.96188055]])

In [ ]:
#to convert vector to probability distribution horizontally so all rows add upto 1 and each prob discribes that word importance iin attention
def softmax(x):
  return (np.exp(x).T / np.sum(np.exp(x), axis=-1)).T

In [ ]:
attention = softmax(scaled + mask)

In [ ]:
attention

array([[1.        , 0.        , 0.        , 0.        ],
       [0.48203493, 0.51796507, 0.        , 0.        ],
       [0.19982158, 0.32351002, 0.4766684 , 0.        ],
       [0.59038094, 0.175298  , 0.10772248, 0.12659857]])

In [ ]:
#multiply value with q.k to get perfect context filled attention vector
new_v = np.matmul(attention, v)
new_v

array([[-0.2684259 ,  0.74669319,  0.25532415,  0.54156151,  0.12737344,
         0.85743961, -0.56087513, -0.44426863],
       [-0.15059575,  1.29280069, -0.02175224, -0.55235642,  1.36705433,
         0.37281449,  0.19175886, -0.01779933],
       [-0.05228277,  0.23047706,  0.06624358, -0.13166569,  0.0531256 ,
         0.91039689, -0.20879857,  0.3477985 ],
       [-0.04263923,  0.55941029,  0.12328911,  0.18090996,  0.37093474,
         0.52739853, -0.47035395, -0.16732325]])

In [ ]:
def softmax(x):
  return (np.exp(x).T / np.sum(np.exp(x), axis=-1)).T

#same above code but for encoder without the mask
def scaled_dot_product_attention(q, k, v, mask=None):
  d_k = q.shape[-1]
  scaled = np.matmul(q, k.T) / math.sqrt(d_k)
  if mask is not None:
    scaled = scaled + mask
  attention = softmax(scaled)
  out = np.matmul(attention, v)
  return out, attention

In [ ]:
values, attention = scaled_dot_product_attention(q, k, v, mask=mask)
print("Q\n", q)
print("K\n", k)
print("V\n", v)
print("New V\n", values)
print("Attention\n", attention)

Q
 [[-1.79708796  0.52122516  0.34239011 -0.2192233   0.95937318  0.01950385
  -0.41948086  0.19684975]
 [-0.28763054  1.56560955 -1.50161045  0.3472905  -0.31556079  1.35173172
  -1.14744159  0.26172031]
 [ 0.49501539  0.83643826 -0.61257021 -0.46052311  0.21198769 -1.21393321
   0.68011574  1.08998828]
 [ 1.04793736  0.90690933  1.12068841  0.96005338 -0.48338012 -0.49387283
   0.89347304 -0.98213068]]
K
 [[-1.69457653  1.14189547 -0.54117259 -0.21483296 -1.06478926 -2.33830904
  -0.53044194 -2.02805418]
 [-0.98502234  0.65551073  0.3332141  -0.5899652  -0.47458067 -1.74058004
  -1.61271634  0.83231212]
 [ 0.5630061  -0.12879049 -0.89508148  1.07619482  2.16600192  0.2716528
  -0.04767808  2.50152317]
 [ 0.13802501  0.90824179 -0.63588164 -2.5203722   0.31566198  0.4199465
  -1.52188367 -1.18428275]]
V
 [[-0.2684259   0.74669319  0.25532415  0.54156151  0.12737344  0.85743961
  -0.56087513 -0.44426863]
 [-0.04093924  1.80102587 -0.27960843 -1.57039158  2.52074117 -0.07819322
   0.892